**Testing Gradient_Boosting_Optimization.py**

In [1]:
#Imports
import pandas as pd
import sys, os, random
import importlib
import numpy as np
from sklearn.metrics import make_scorer, recall_score
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

#Class Import
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../.."))) #Allow for imports from src
from src.models import Gradient_Boosting_Optimization
importlib.reload(Gradient_Boosting_Optimization) #Ensures file is uptodate!
from src.models.Gradient_Boosting_Optimization import Gradient_Boosting_Optimization

#Set Seed
os.environ["PYTHONHASHSEED"] = "1945"
random.seed(1945)
np.random.seed(1945)

In [2]:
#Complied data of convoys
#Routes examined are HX, SC, OB, ON, ONS
df = pd.read_csv('/Users/matthewplambeck/Desktop/Convoy Predictor/data/processed/Complete_Convoy_Data.csv')
df = df.drop(columns=['Unnamed: 0'])
df.shape #Test

(1174, 21)

In [3]:
#Drop unecessary/redundent features
df = df.drop(columns=['Convoy Number', 'Number of Ships Sunk', 'Depart_Date', 'Arrival/Dispersal Date', 'Number of Escorts Sunk', \
                         'Number of Stragglers Sunk', 'Total Tons of Ships Sunk', 'Escort Sink Percentage', 'Straggler Sink Percentage'])
df.reset_index(drop=True).head(3)

,Number of Ships,Number of Escort Ships,Number of Stragglers,Total Tons of Convoy,Overall Sink Percentage,Avg Number of U-Boats in Atlantic,Escort Ratio,Time At Sea (Days),Month,Year,Previous Month Avg Sink %,Approx. Sighting Range
0,5.0,2.0,0.0,22877.0,0.0,6.0,0.400000,3.0,9.0,1939.0,0.0,12.190200
1,5.0,2.0,0.0,22967.0,0.0,6.0,0.400000,5.0,9.0,1939.0,0.0,12.190200
2,7.0,4.0,0.0,21293.0,0.0,6.0,0.571429,5.0,9.0,1939.0,0.0,14.434062


In [4]:
#Convert Overall Sink Percentage to binary 1( High)
df['Risk'] = (df['Overall Sink Percentage'] > 0).astype(int) 
#Risk is binary based off whether a ship was sunk while in a convoy:  (0 = No Ships Sunk, 1 = At Least One Ship Sunk)
X = np.array(df.drop(columns=['Overall Sink Percentage', 'Risk'])) #Remove Overall Sink Percentage as it leaks data
y = df['Risk'].values #Prediction value

In [9]:
# Baseline ML_Class_1

print("Legacy (ML_Class_1.py) GB Test Results")

#Class Import
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../.."))) #Allow for imports from src
from src.models import ML_Class_1
importlib.reload(ML_Class_1) #Ensures file is uptodate!
from src.models.ML_Class_1 import Model_Tester as LegacyTester

legacy = LegacyTester(
    model=GradientBoostingClassifier(random_state=1945),
    parameter_grid=None,
    cv_folds=5)

legacy.train_test_split(X, y, train_size=0.8, random_state=1945)
legacy.optimize()  #no grid search
legacy_results = legacy.evaluate(show_plots=False)
print("Legacy recall:", legacy_results["classification_report"]["1"]["recall"])


Legacy (ML_Class_1.py) GB Test Results

GradientBoostingClassifier Evaluation:

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.95      0.90       185
           1       0.71      0.44      0.54        50

    accuracy                           0.84       235
   macro avg       0.79      0.70      0.72       235
weighted avg       0.83      0.84      0.83       235


ROC AUC Score: 0.8351
Matthews Correlation Coefficient (MCC): 0.4733
Balanced Accuracy: 0.6957
GradientBoostingClassifier Confusion Matrix (values only):
[[176   9]
 [ 28  22]]
Legacy recall: 0.44


In [10]:
#Gradient_Boosting_Optimization, default GB (no grid or threshold) Used as a compairson to the Legacy tester
gb_default = Gradient_Boosting_Optimization(
    model=GradientBoostingClassifier(random_state=1945),
    parameter_grid=None,
    cv_folds=5,
    positive_label=1,
    optimize_scoring="recall",
    auto_calibrate_threshold=False)  #No threshold optimization


gb_default.train_test_split(X, y, train_size=0.8, random_state=1945)
gb_default.optimize()
gb_default_results = gb_default.evaluate(show_plots=False)


Applied decision threshold: 0.5000

GradientBoostingClassifier Evaluation:

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.95      0.90       185
           1       0.71      0.44      0.54        50

    accuracy                           0.84       235
   macro avg       0.79      0.70      0.72       235
weighted avg       0.83      0.84      0.83       235


ROC AUC Score: 0.8351
Matthews Correlation Coefficient (MCC): 0.4733
Balanced Accuracy: 0.6957
Confusion Matrix:
          Predicted 0  Predicted 1
Actual 0          176            9
Actual 1           28           22
Recall (positive=1): 0.4400
F2 Score: 0.4762
False Negatives: 28


**Results for both are the same which indicates both are working as intended**

**Simple Optimization Tests:**

In [11]:
#Gradient_Boosting_Optimization, grid-search & F2 threshold optimization
gb_grid = Gradient_Boosting_Optimization(
    model=GradientBoostingClassifier(random_state=1945),
    parameter_grid={
        "learning_rate": [0.09, 0.1, 0.11],
        "n_estimators": [340, 345, 350,],
        "max_depth": [4, ],
        "min_samples_leaf": [1, 3, 5],
        "subsample": [0.8, 1.0]},
    cv_folds=5,
    positive_label=1,
    optimize_scoring="recall",
    auto_calibrate_threshold=True,
    threshold_beta=2.0)

gb_grid.train_test_split(X, y, train_size=0.8, random_state=1945)
gb_grid.optimize()
gb_grid_results = gb_grid.evaluate(show_plots=False)



Best Hyperparameters Found:
{'learning_rate': 0.09, 'max_depth': 4, 'min_samples_leaf': 3, 'n_estimators': 350, 'subsample': 1.0}
Best Cross-Validation Score: 0.4480
Applied decision threshold: 0.5906 (F-beta: 1.0000)

GradientBoostingClassifier Evaluation:

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.95      0.90       185
           1       0.69      0.44      0.54        50

    accuracy                           0.84       235
   macro avg       0.77      0.69      0.72       235
weighted avg       0.82      0.84      0.82       235


ROC AUC Score: 0.7955
Matthews Correlation Coefficient (MCC): 0.4605
Balanced Accuracy: 0.6930
Confusion Matrix:
          Predicted 0  Predicted 1
Actual 0          175           10
Actual 1           28           22
Recall (positive=1): 0.4400
F2 Score: 0.4741
False Negatives: 28


In [12]:
print("Recall Compairsons: \n")
print("Baseline recall:", legacy_results["classification_report"]["1"]["recall"])
print("Default GB recall:", gb_default_results["classification_report"]["1"]["recall"])
print("Grid-search recall:", gb_grid_results["recall"])
# print("Grid search best params:", gb_grid.best_model.get_params())
# print("Grid search decision threshold:", gb_grid_results["decision_threshold"])

Recall Compairsons: 

Baseline recall: 0.44
Default GB recall: 0.44
Grid-search recall: 0.44


In [21]:
#Manual Threshold Comparison using gb_grid: 
thresholds = np.arange(0, 1, 0.05) 
thresholds = thresholds.tolist() 
rows = []

for t in thresholds:
    gb_grid.set_decision_threshold(t)
    res = gb_grid.evaluate(show_plots=False, print_results=False)
    rows.append({
        "threshold": t,
        "recall": res["recall"],
        "precision": res["classification_report"]["1"]["precision"],
        "false_negatives": res["false_negatives"],
        "f1-scores": res["classification_report"]["macro avg"]["f1-score"]
        })
gb_grid.set_decision_threshold(None) 
threshold_results = pd.DataFrame(rows)
threshold_results

/opt/homebrew/Caskroom/miniforge/base/envs/Python-ML/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/homebrew/Caskroom/miniforge/base/envs/Python-ML/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/homebrew/Caskroom/miniforge/base/envs/Python-ML/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control thi

,threshold,recall,precision,false_negatives,f1-scores
0,0.00,1.00,0.212766,0,0.175439
1,0.05,0.82,0.376147,9,0.634067
2,0.10,0.74,0.451220,13,0.694504
3,0.15,0.70,0.507246,15,0.724317
4,0.20,0.64,0.551724,18,0.735523
5,0.25,0.60,0.600000,20,0.745946
6,0.30,0.56,0.608696,22,0.738191
7,0.35,0.48,0.571429,26,0.702669
8,0.40,0.48,0.615385,26,0.715857
9,0.45,0.48,0.631579,26,0.720371


**XGBoost Tests:**

In [14]:
#TODO Analyze Results
xgb_params = {"eta": [0.05, 0.1],"max_depth": [3, 6],"subsample": [0.8, 1.0],"colsample_bytree": [0.7, 1.0],"gamma": [0, 1],
    "scale_pos_weight": [1, 3],}

xgb_optimizer = Gradient_Boosting_Optimization(
    model=XGBClassifier(
        objective="binary:logistic",
        eval_metric="logloss",
        use_label_encoder=False,
        random_state=1945,
        n_estimators=500),
    parameter_grid=xgb_params,
    positive_label=1,
    optimize_scoring="recall",
    auto_calibrate_threshold=True,
    threshold_beta=2.0,)

xgb_optimizer.train_test_split(X, y, train_size=0.8, random_state=1945)
xgb_optimizer.optimize(
    fit_params={"eval_set": [(xgb_optimizer.X_test, xgb_optimizer.y_test)], "verbose": False,})
xgb_results = xgb_optimizer.evaluate(show_plots=False)

/opt/homebrew/Caskroom/miniforge/base/envs/Python-ML/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [15:06:23] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1755048541311/work/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/homebrew/Caskroom/miniforge/base/envs/Python-ML/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [15:06:23] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1755048541311/work/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/homebrew/Caskroom/miniforge/base/envs/Python-ML/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [15:06:23] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1755048541311/work/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/homebrew/Caskroom/minif


Best Hyperparameters Found:
{'colsample_bytree': 0.7, 'eta': 0.1, 'gamma': 1, 'max_depth': 3, 'scale_pos_weight': 3, 'subsample': 1.0}
Best Cross-Validation Score: 0.6070
Applied decision threshold: 0.4420 (F-beta: 0.8410)

XGBClassifier Evaluation:

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.84      0.87       185
           1       0.53      0.66      0.59        50

    accuracy                           0.80       235
   macro avg       0.72      0.75      0.73       235
weighted avg       0.82      0.80      0.81       235


ROC AUC Score: 0.8152
Matthews Correlation Coefficient (MCC): 0.4673
Balanced Accuracy: 0.7516
Confusion Matrix:
          Predicted 0  Predicted 1
Actual 0          156           29
Actual 1           17           33
Recall (positive=1): 0.6600
F2 Score: 0.6298
False Negatives: 17
